# libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import time
import scipy.optimize as optimize
import lightgbm as lgb
from bs4 import BeautifulSoup


from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from scipy.stats import rankdata

from collections import defaultdict
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator
from tqdm import tqdm

import time
import re 
import scipy
from scipy import sparse
import gc 
from IPython.display import display, HTML
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

In [2]:
#1 means kf 2 means sample
Fold_type=2
is_test=False
translate_aug=False
#记得改kdict 里面的tranlate的值


In [3]:
fold_num_k=5
fold_num_s=1

In [4]:
# data_names=["jc_","juc_","rud_","jcc_"]
# model_choice=["ridge","gbm"]
# factor=[0.5,0.5]

data_names=["jcc_"]
translate_data=["jc_"]

model_choice=["ridge"]
factor=[1]

In [5]:
system_path=r"C:\Users\Lenovo\Desktop\stupidcode\data\jigsaw"
# model_all=["ridge","gbm"]
# for model_name in model_all:
#     if model_name not in os.listdir():
#         os.makedirs(f"{model_name}")
out_path=r"C:\Users\Lenovo\Desktop\stupidcode\data\jigsaw\save_model\temp_work_save"

# Data

In [6]:
#第一届 jigsaw比赛 数据（challenge） Toxic Comment Classification Challenge

jc_path=os.path.join(system_path,"jigsaw-toxic-comment-classification-challenge")
jc_trans_path=os.path.join(system_path,"jigsaw-toxic-comment-classification-challenge")
#ruddit 数据
run_path=os.path.join(system_path,"ruddit-jigsaw-dataset/Dataset")
#第二届 jigsaw比赛 对少数人群不歧视
juc_path=os.path.join(system_path,"jigsaw-unintended-bias-in-toxicity-classification")

#本次比赛数据 作为val
jts_path=os.path.join(system_path,"jigsaw-toxic-severity-rating")

# #数据抽样存储路径
gbm_save_path=os.path.join(out_path,"gbm")
ridge_save_path=os.path.join(out_path,"ridge")

导入数据

In [7]:
#验证集和测试集
df_val = pd.read_csv(os.path.join(jts_path,"validation_data.csv"))

df_test = pd.read_csv(os.path.join(jts_path,"comments_to_score.csv"))

In [8]:
#第一届比赛数据 以0/1为分值 
features = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

jc_train_df = pd.read_csv(os.path.join(jc_path,"train.csv"))
# jc_test_df = pd.read_csv(os.path.join(jc_path,"test.csv"))
# temp_df = pd.read_csv(os.path.join(jc_path,"test_labels.csv"))

# jc_test_df = jc_test_df.merge ( temp_df, on ="id")
# #drop test data not used for scoring
# jc_test_df = jc_test_df.query ("toxic != -1")
# jc_df = jc_train_df.append ( jc_test_df ) 

# print(f"Train+Test:{jc_df.shape[0]}")

jc_df=jc_train_df

# 将代表有毒行为的筛选出来
jc_df["toxic_subtype_sum"]=jc_df[features].sum(axis=1)
jc_df["toxic_behaviour"]=jc_df["toxic_subtype_sum"].map(lambda x: x > 0)

tot_toxic_behaviour = jc_df["toxic_behaviour"].sum()
print(f'comments with toxic behaviour:{tot_toxic_behaviour}')
jc_df=jc_df.reset_index(drop=True)

comments with toxic behaviour:16225


In [9]:
#第一届比赛 数据预处理
# toxic = 1.0
# severe_toxic = 2.0
# obscene = 1.0
# threat = 1.0
# insult = 1.0
# identity_hate = 2.0
toxic = 0.32
severe_toxic = 1.5
obscene = 0.16
threat = 1.5
insult = 0.64
identity_hate = 1.5

def create_train (df):
#     df['y'] = df[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].max(axis=1)
    df['y'] = df['toxic']*toxic
    df['y'] = df["y"]+df['severe_toxic']*severe_toxic
    df['y'] = df["y"]+df['obscene']*obscene
    df['y'] = df["y"]+df['threat']*threat
    df['y'] = df["y"]+df['insult']*insult
    df['y'] = df["y"]+df['identity_hate']*identity_hate
    
    df = df[["id",'comment_text', 'y', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].rename(columns={'comment_text': 'text'})
    return df
        
jc_df = create_train (jc_df)
jc_df.shape

(159571, 9)

In [10]:
# 第二届比赛 以打分形式0~1
features = ["toxicity","severe_toxicity","obscene","insult","identity_attack", "sexual_explicit"]
cols = ['id', 'comment_text', 'toxicity', 'severe_toxicity', 'obscene', 'threat','insult', 'identity_attack', 'sexual_explicit', 'toxicity_annotator_count']

juc_df = pd.read_csv(os.path.join(juc_path,"all_data.csv"))

juc_df = juc_df.query ("toxicity_annotator_count > 5")
print(f"juc_df:{juc_df.shape}")
juc_df['y'] = juc_df[[ 'severe_toxicity', 'obscene', 'sexual_explicit','identity_attack', 'insult', 'threat']].sum(axis=1)

#毒性<0.5 按标注 否则算所有有毒行为的标注和
juc_df['y'] = juc_df.apply(lambda row: row["toxicity"] if row["toxicity"] <= 0.5 else row["y"] , axis=1)
juc_df = juc_df[['comment_text', 'y']].rename(columns={'comment_text': 'text'})
# min_len = (juc_df['y'] > 0.5).sum()


juc_df:(601044, 46)


In [11]:
#ruddit 数据
rud_df = pd.read_csv(os.path.join(run_path,"ruddit_with_text.csv"))

print(f"rud_df:{rud_df.shape}")
rud_df['y'] = rud_df['offensiveness_score'].map(lambda x: 0.0 if x <=0 else x)
rud_df = rud_df[['txt', 'y']].rename(columns={'txt': 'text'})

rud_df:(5838, 5)


In [12]:
#clean data
def clean(data, col):
    #数据清洗 
    
    # Clean some punctutations
    data[col] = data[col].str.replace('\n', ' \n ')
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3')
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1')    
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\']+)',r' \1 ')    
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1')
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1')
    data[col] = data[col].str.replace(r'[ ]{2,}',' ').str.strip()   
    
    return data

def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

#第一届大赛 clean 数据
# jcc_df=clean(jc_df,"text")

tqdm.pandas()
jcc_df=jc_df.copy()
# jcc_df['text'] = jcc_df.apply(lambda row:text_cleaning(row["text"]),axis=1)

jcc_df['text']=jcc_df['text'].progress_apply(text_cleaning)

100%|████████████████████████████████████████████████████████████████████████| 159571/159571 [01:02<00:00, 2537.52it/s]


In [13]:
jcc_df.head()

,id,text,y,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation Why the edits made under my userna...,0.0,0,0,0,0,0,0
1,000103f0d9cfb60f,D aww He matches this background colour I m se...,0.0,0,0,0,0,0,0
2,000113f07ec002fd,Hey man I m really not trying to edit war It s...,0.0,0,0,0,0,0,0
3,0001b41b1c6bb37e,More I can t make any real suggestions on impr...,0.0,0,0,0,0,0,0
4,0001d958c54c6e35,You sir are my hero Any chance you remember wh...,0.0,0,0,0,0,0,0


# 消重

## 去关键字

### aug_trans

In [14]:
if translate_aug==True:
    jc_trans=pd.read_csv(os.path.join(jc_trans_path,"jc_trans.csv"))
    jc_trans.set_index(["Unnamed: 0"], inplace=True)

    jc_df=pd.merge(jc_df,jc_trans[["id","fr_text"]],on="id",how="left")
    jc_df=pd.merge(jc_df,jc_trans[["id","es_text"]],on="id",how="left")
    jc_df=pd.merge(jc_df,jc_trans[["id","de_text"]],on="id",how="left")

# create fold

In [15]:
def create_fold_kfold(df,FOLDS=5,select_num=0.5,frac_1=1.5,balance=False):
    #select num challenge:1 unbias:0.5  

    min_len = (df['y'] >= select_num).sum()
    if balance==False:
        #采样负样本
        df_y0_undersample = df[df['y'] <select_num].sample(n=int(min_len*frac_1),random_state=201)
        df = pd.concat([df[-(df['y'] <select_num)], df_y0_undersample])
        df=df.reset_index(drop=True)
    y=df["y"].values
    x=df["text"]
    stratified = np.around (y)
    
    kf = StratifiedKFold(n_splits=FOLDS,shuffle=True,random_state=123)
    df_dict={}
    for fold,(train_index,val_index) in enumerate(kf.split(x,stratified)):
        df_train=df.iloc[train_index]
        df_dict[fold]=df_train
    
    return df_dict

def create_fold_sample(df,n_folds=3,frac_1=0.8,frac_1_factor=1.5,select_num=0,balance=False,translate=False):
    df_dict={}
    #正样本 大于等于select_num
    select_list=-(df['y'] <select_num)
    min_len=select_list.sum()
    if translate==True:
        min_len=4*min_len
    for fld in range(n_folds):
        if balance==False:
#             df_y0_undersample=df[df.y<select_num].sample(n=int(min_len*frac_1*frac_1_factor), random_state = 10*(fld+1))
            df_y0_undersample=df[df.y<select_num].sample(n=int(min_len*frac_1*frac_1_factor), random_state = 201)

            tmp_df = pd.concat([df[select_list].sample(frac=frac_1, random_state = 10*(fld+1)),df_y0_undersample ])
        else:
            tmp_df = df.sample(frac=frac_1, random_state = 10*(fld+1))
            
        df_dict[fld]=tmp_df

    return df_dict


In [16]:
if is_test==True:
    jc_df=jc_df[0:400]
    juc_df=juc_df[0:400]
    rud_df=rud_df[0:400]
    jcc_df=jcc_df[0:400]

In [17]:
if is_test==True:
#     jc_df_kdict=create_fold_kfold(jc_df,FOLDS=fold_num_k,select_num=1,frac_1=1.5,balance=True)
#     juc_df_kdict=create_fold_kfold(juc_df,FOLDS=fold_num_k,select_num=0.5,frac_1=1.5,balance=True)
#     rud_df_kdict=create_fold_kfold(rud_df,FOLDS=fold_num_k,select_num=0.5,frac_1=1.5,balance=True)
#     jcc_df_kdict=create_fold_kfold(jcc_df,FOLDS=fold_num_k,select_num=1,frac_1=1.5,balance=True)

    jc_df_sdict=create_fold_sample(jc_df,n_folds=fold_num_s,frac_1=0.8,frac_1_factor=1.5,select_num=1,balance=True,translate=False)
    juc_df_sdict=create_fold_sample(juc_df,n_folds=fold_num_s,frac_1=0.8,frac_1_factor=1.5,select_num=0.5,balance=True)
    rud_df_sdict=create_fold_sample(rud_df,n_folds=fold_num_s,frac_1=0.8,frac_1_factor=1.5,select_num=0.5,balance=True)
    jcc_df_sdict=create_fold_sample(jcc_df,n_folds=fold_num_s,frac_1=0.8,frac_1_factor=1.5,select_num=1,balance=True)
    
if is_test==False:
#     jc_df_kdict=create_fold_kfold(jc_df,FOLDS=fold_num_k,select_num=1,frac_1=1.5,balance=False)
#     juc_df_kdict=create_fold_kfold(juc_df,FOLDS=fold_num_k,select_num=0.5,frac_1=1.5,balance=False)
#     rud_df_kdict=create_fold_kfold(rud_df,FOLDS=fold_num_k,select_num=0.5,frac_1=1.5,balance=True)
#     jcc_df_kdict=create_fold_kfold(jcc_df,FOLDS=fold_num_k,select_num=1,frac_1=1.5,balance=False)

    jc_df_sdict=create_fold_sample(jc_df,n_folds=fold_num_s,frac_1=0.8,frac_1_factor=1.5,select_num=0.001,balance=False,translate=False)
    juc_df_sdict=create_fold_sample(juc_df,n_folds=fold_num_s,frac_1=0.8,frac_1_factor=1.5,select_num=0.5,balance=False)
    rud_df_sdict=create_fold_sample(rud_df,n_folds=fold_num_s,frac_1=0.8,frac_1_factor=1.5,select_num=0.5,balance=True)
    jcc_df_sdict=create_fold_sample(jcc_df,n_folds=fold_num_s,frac_1=1,frac_1_factor=1,select_num=0.001,balance=False)
    
#     jc_df_kdict[0].shape
#     juc_df_kdict[0].shape
#     rud_df_kdict[0].shape
#     jcc_df_sdict[0].shape

In [18]:
jcc_df_sdict[0]["y"].value_counts()

0.00    16225
0.32     5666
1.12     3800
0.48     1758
2.62     1738
0.96     1215
4.12      385
0.16      317
0.64      301
1.82      290
1.98      204
0.80      181
2.46      164
1.50       76
5.62       31
2.14       31
3.32       21
2.30       20
3.96       10
3.48       10
1.66        5
4.82        1
3.96        1
Name: y, dtype: int64

# ridge

In [19]:
import joblib
def ridge_cv(df_dic,n_folds,model_pre="jc_k_ridge_",df_val=df_val,clean_prm=False,translate=False):
    val_preds_arr1 = np.zeros((df_val.shape[0], n_folds))
    val_preds_arr2 = np.zeros((df_val.shape[0], n_folds))
    test_preds_arr = np.zeros((df_test.shape[0], n_folds))
    for fld in tqdm(range(n_folds)):
        df = df_dic[fld]
        vec = TfidfVectorizer(analyzer='char_wb', max_df=0.5, min_df=3, ngram_range=(3, 5) )
        vec_pre=model_pre+"vec_"
        if translate!=True:
            text=df["text"]
            y=df["y"]
        else:
            trans_df=df.dropna(axis=0,subset = ["fr_text"])
            text=pd.concat([df["text"],trans_df["fr_text"],trans_df["es_text"],trans_df["de_text"]])
            y=pd.concat([df["y"],trans_df["y"],trans_df["y"],trans_df["y"]])
        X=vec.fit_transform(text)
        joblib.dump(vec,os.path.join(ridge_save_path,f'{vec_pre}{fld}.pkl')) #保存模型 文件后缀为.pkl
        model=Ridge(alpha=0.5)
        model.fit(X,y)

        if clean_prm==True:
#             X_less_toxic = vec.transform(clean(df_val,'less_toxic')['less_toxic'])
#             X_more_toxic = vec.transform(clean(df_val,'more_toxic')['more_toxic'])
#             X_test = vec.transform(clean(df_test,'text')['text'])
            df_val=clean(df_val,'less_toxic')
            df_val=clean(df_val,'more_toxic')
#             df_test=clean(df_test,"text")
            df_val['less_toxic'] = df_val.apply(lambda row :text_cleaning(row["less_toxic"]),axis=1)
            df_val['more_toxic'] = df_val.apply(lambda row :text_cleaning(row["more_toxic"]),axis=1)
            df_test['text'] = df_test.apply(lambda row :text_cleaning(row["text"]),axis=1)
        
    
#             df_val['less_toxic'] = df_val['less_toxic'].progress_apply(text_cleaning)
#             df_val['more_toxic'] = df_val['more_toxic'].progress_apply(text_cleaning)
#             df_test['text'] = df_test['text'].progress_apply(text_cleaning)

            X_less_toxic = vec.transform(df_val['less_toxic'])
            X_more_toxic = vec.transform(df_val['more_toxic'])
            X_test = vec.transform(df_test['text'])
        else:
            X_less_toxic = vec.transform(df_val['less_toxic'])
            X_more_toxic = vec.transform(df_val['more_toxic'])
            X_test = vec.transform(df_test['text'])
            
        val_preds_arr1[:,fld] = model.predict(X_less_toxic)
        val_preds_arr2[:,fld] = model.predict(X_more_toxic)

        test_preds_arr[:,fld] = model.predict(X_test)
            
        joblib.dump(model,os.path.join(ridge_save_path,f'{model_pre}{fld}.pkl')) #保存模型 文件后缀为.pkl
        del model,vec
    p1=val_preds_arr1.mean(axis=1)
    p2=val_preds_arr2.mean(axis=1)
    pv=test_preds_arr.mean(axis=1)
    print(f'Validation Accuracy is { np.round((p1 < p2).mean() * 100,2)}')        
    return p1,p2,pv

# lightgbm

In [20]:
#verbose =-1 忽略警告
depth=1000
params = {'task':'train',
    "device" : "cpu",
    'boosting_type':'gbdt',
    "max_depth":7,
    "num_leaves":80,
    "bagging_fraction":0.8,
          "bagging_freq":5,
    'objective':'regression',
    "learning_rate":0.05,
    "max_bin":63,
    "random_state":2021,
    'verbose':-1, 
    "train_metric":False,
    "force_col_wise":False}


def lightgbm_cv(df_dic,n_folds,model_pre="jc_k_gbm_",df_val=df_val,clean_prm=False,translate=False):
    ####lightgbm_cv 在aug情况下有bug
    val_preds_arr1 = np.zeros((df_val.shape[0], n_folds))
    val_preds_arr2 = np.zeros((df_val.shape[0], n_folds))
    test_preds_arr = np.zeros((df_test.shape[0], n_folds))

    for fld in tqdm(range(n_folds)):
        df = df_dic[fld]
        vec = TfidfVectorizer(analyzer='char_wb', max_df=0.5, min_df=3, ngram_range=(3, 5) )
        if translate!=True:
            text=df["text"]
            y=df["y"]
        else:
            trans_df=df.dropna(axis=0,subset = ["fr_text"])
            text=pd.concat([df["text"],trans_df["fr_text"],trans_df["es_text"],trans_df["de_text"]])
            y=pd.concat([df["y"],trans_df["y"],trans_df["y"],trans_df["y"]])

        X=vec.fit_transform(text)
        vec_pre=model_pre+"vec_"
        joblib.dump(vec,os.path.join(gbm_save_path,f'{vec_pre}{fld}.pkl')) #保存模型 文件后缀为.pkl
        X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.1,random_state = 0)
        lgb_train = lgb.Dataset(X_train,y_train)
        lgb_eval = lgb.Dataset(X_val,y_val,reference=lgb_train)
        
        gbm = lgb.train(params,lgb_train,
            num_boost_round=300,
            valid_sets=lgb_eval,
            early_stopping_rounds=50
            ) 
        
        if clean_prm==True:
            X_less_toxic = vec.transform(clean(df_val,'less_toxic')['less_toxic'])
            X_more_toxic = vec.transform(clean(df_val,'more_toxic')['more_toxic'])
            X_test = vec.transform(clean(df_test,'text')['text'])
        else:
            X_less_toxic = vec.transform(df_val['less_toxic'])
            X_more_toxic = vec.transform(df_val['more_toxic'])
            X_test = vec.transform(df_test['text'])
    
        val_preds_arr1[:,fld] = gbm.predict(X_less_toxic,num_iteration=gbm.best_iteration)
        val_preds_arr2[:,fld] = gbm.predict(X_more_toxic,num_iteration=gbm.best_iteration)

        test_preds_arr[:,fld] =gbm.predict(X_test,num_iteration=gbm.best_iteration)
        
        gbm.save_model(os.path.join(gbm_save_path,f'{model_pre}{fld}.txt'))
        # 模型加载
#         gbm = lgb.Booster(model_file='model.txt')
        # 模型预测
#         y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
        del gbm,vec
    p1=val_preds_arr1.mean(axis=1)
    p2=val_preds_arr2.mean(axis=1)
    pv=test_preds_arr.mean(axis=1)
    print(f'Validation Accuracy is { np.round((p1 < p2).mean() * 100,2)}')        
    return p1,p2,pv

In [21]:
p1=defaultdict()
p2=defaultdict()
pv=defaultdict()


func_dict={"ridge":ridge_cv,"gbm":lightgbm_cv}
# func_dict.get(x)

if Fold_type==1:
#     pre_names=["jc_k_","juc_k_","rud_k_","jcc_k_"]
    pre_names=[ data_name+"k_" for data_name in data_names]
    name2dict={"jc_k_":jc_df_kdict,"juc_k_":juc_df_kdict,"rud_k_":rud_df_kdict,"jcc_k_":jcc_df_kdict}
    fold_num=fold_num_k
elif Fold_type==2:
    pre_names=[ data_name+"s_" for data_name in data_names]
    name2dict={"jc_s_":jc_df_sdict,"juc_s_":juc_df_sdict,"rud_s_":rud_df_sdict,"jcc_s_":jcc_df_sdict}
    fold_num=fold_num_s

p1_ensenmble = np.zeros((df_val.shape[0]))
p2_ensenmble = np.zeros((df_val.shape[0]))
score=np.zeros((df_test.shape[0]))

for pre_name in pre_names:
    ###model_pre_ridge:jc_k_ridge_ pre_name:jc_k_ model:jc_k_ridge_{fold} vec:jc_k_ridge_vec_{fold}
    #pre_name jc_k_ model_name jc_k_ridge_
    clean_prm=False
    translate=False
    p1[pre_name],p2[pre_name]=np.zeros((df_val.shape[0])),np.zeros((df_val.shape[0]))
    pv[pre_name]=np.zeros((df_test.shape[0]))
    if "jcc" in pre_name:
        clean_prm=True
    if any([ trans_name in pre_name for trans_name in translate_data]) and translate_aug==True:
        translate=True

    for index,model_name in enumerate(model_choice):
        cv_func=func_dict.get(model_name)
        model_pre=pre_name+model_name+"_"
        p1[model_pre],p2[model_pre],pv[model_pre]=cv_func(name2dict[pre_name],n_folds=fold_num,
                                                        model_pre=model_pre,clean_prm=clean_prm,translate=translate)

        p1[pre_name]= p1[pre_name]+ p1[model_pre]*factor[index]
        p2[pre_name]= p2[pre_name]+ p2[model_pre]*factor[index]
        pv[pre_name]= pv[pre_name]+ pv[model_pre]*factor[index]

    kmax=max(p1[pre_name].max(),p2[pre_name].max())
    p1_ensenmble=p1_ensenmble+p1[pre_name]/kmax
    p2_ensenmble=p2_ensenmble+p2[pre_name]/kmax
    score=score+pv[pre_name]/kmax

print(f' Validation Accuracy is { np.round((p1_ensenmble < p2_ensenmble).mean() * 100,4)}') 

    
df_test['score'] = rankdata(score, method='ordinal')


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:16<00:00, 76.50s/it]

Validation Accuracy is 67.05
 Validation Accuracy is 67.0519


In [22]:
df_test[['comment_id', 'score']].to_csv("submission.csv", index=False)